In [1]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import torch


In [2]:
ratings = pd.read_csv('data/The Movies Dataset/ratings_small.csv')
movies = pd.read_csv('data/The Movies Dataset/movies_metadata.csv')

/var/folders/lt/jr5mpz990s58yvgvt5_n2d800000gn/T/ipykernel_46272/1917082022.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('data/The Movies Dataset/movies_metadata.csv')


In [3]:
ratings.drop(['timestamp'], axis=1, inplace=True)

In [4]:
movies.rename(columns={'id': 'movieId'}, inplace=True)

In [5]:
movies.drop(['homepage', 'imdb_id'], axis=1, inplace=True)

In [6]:
ratings['movieId'] = ratings['movieId'].astype(str)

In [7]:
ratings_movies = pd.merge(ratings, movies, on='movieId')

In [8]:
ratings_movies = ratings_movies.filter(['userId', 'movieId', 'rating'])
ratings_movies.head()

,userId,movieId,rating
0,1,1371,2.5
1,4,1371,4.0
2,7,1371,3.0
3,19,1371,4.0
4,21,1371,3.0


In [9]:
#user-movie matrix
users_movies = ratings_movies.pivot_table(index='userId', columns = 'movieId', values = 'rating').fillna(0)

In [10]:
users_movies.head()

movieId,100,100017,100032,100272,100450,101,101362,1018,101904,102,102165,1023,1024,103,103048,103210,103299,103502,103539,103688,103731,104,1040,104211,104241,104374,104863,1049,105,1050,1051,105121,105246,105254,105593,105763,1059,106417,107,107081,...,956,957,95744,95752,95949,95963,96,961,962,963,964,965,966,968,96821,97,97593,976,97936,97938,98,980,981,98122,982,98369,984,98491,986,98604,987,988,99,990,991,99106,992,994,996,99846
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
users_movies_np = users_movies.values
user_num = users_movies.shape[0]
item_num = users_movies.shape[1]     

In [12]:
#rating이 되어 있는 요소들의 row와 col
features = np.transpose(users_movies_np.nonzero()).tolist()

In [13]:
labels = []
for i, j in features:
    labels.append(users_movies_np[i, j])
     

In [14]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2)

In [15]:
class MLP_MF(torch.nn.Module):
    def __init__(self, user_num, item_num, factor_num, num_layers = 3):
        super(MLP_MF, self).__init__()

        # 유저 임베딩
        self.user_embedding = torch.nn.Embedding(user_num, factor_num)

        # 아이템 임베딩
        self.item_embedding = torch.nn.Embedding(item_num, factor_num)

        MLP_modules = []
        input_size = factor_num * 2
        for i in range(num_layers):
            MLP_modules.append(torch.nn.Linear(input_size, input_size // 2))
            MLP_modules.append(torch.nn.ReLU())
            input_size //=  2
        self.MLP_layers = torch.nn.Sequential(*MLP_modules)

        # FC
        self.FC_layer = torch.nn.Sequential(
            torch.nn.Linear(input_size, 1)
        )
        self._init_weight_()

    def _init_weight_(self):
      # weight 초기화
        torch.nn.init.xavier_uniform_(self.user_embedding.weight)
        torch.nn.init.xavier_uniform_(self.item_embedding.weight)
        for m in self.MLP_layers:
            if isinstance(m, torch.nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)

        for m in self.FC_layer:
            if isinstance(m, torch.nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight)

    def forward(self, user, item):
        user_embedding = self.user_embedding(user)
        item_embedding = self.item_embedding(item)

        concat_two_latent_vactors = torch.cat((user_embedding, item_embedding), -1)
        output_MLP = self.MLP_layers(concat_two_latent_vactors)

        out = self.FC_layer(output_MLP)
        out = out.view(-1)

        return out

In [16]:
batch_size = 16
epochs = 100

In [17]:
train = TensorDataset(torch.LongTensor(X_train),torch.FloatTensor(y_train))
train_dataloader = DataLoader(train, batch_size = batch_size, shuffle=True)
test = TensorDataset(torch.LongTensor(X_test), torch.FloatTensor(y_test))
test_dataloader = DataLoader(test, batch_size = batch_size, shuffle = True)

In [20]:
model = MLP_MF(user_num = user_num, item_num = item_num, factor_num = 20)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [21]:
#training
for epoch in range(1, epochs + 1):
    train_loss = 0
    for data in train_dataloader:
        x, label = data
        user = x[:,0]
        item = x[:,1]
        optimizer.zero_grad()
        output = model(user, item)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_dataloader)
    if epoch % 20 == 0:
        print(f'Epoch: {epoch}, Train Loss: {train_loss:.4f}')

Epoch: 20, Train Loss: 0.4485
Epoch: 40, Train Loss: 0.3600
Epoch: 60, Train Loss: 0.3174
Epoch: 80, Train Loss: 0.2918
Epoch: 100, Train Loss: 0.2727


In [22]:
#test
import math
total_loss = 0
num_samples = 0

with torch.no_grad():
    for data in test_dataloader:
        x, label = data
        user = x[:,0]
        item = x[:,1]
        output = model(user, item)
        total_loss += loss.item() * user.size(0)
        num_samples += user.size(0)
rmse = math.sqrt(total_loss / num_samples)

print(f'Test RMSE: {rmse:.4f}')


Test RMSE: 0.2619


In [43]:
#특정 유저의 예상 평점이 가장 높은 top k개 movie id를 return
def pred_user_i(users_movies_df,users_movies_np,user_id,top_k = 5):
    
    user_id -= 1
    user_rows = users_movies_np[user_id].copy()
    rated_idx = user_rows != 0
    for i in np.where(user_rows == 0)[0]:
        values = model(torch.LongTensor([user_id]), torch.LongTensor([i])).detach().numpy()[0]
        user_rows[i] = values
    user_rows[rated_idx] = -np.inf
    movie_idx = np.argpartition(user_rows, -top_k)[-top_k:].tolist()
    pred_rating = user_rows[movie_idx]
    movie_idx = users_movies.iloc[:,movie_idx].columns.tolist()   
    return pred_rating, movie_idx

In [46]:
pred_rating, movieid_list = pred_user_i(users_movies, users_movies_np, user_id = 1)

In [47]:
movies[movies['movieId'].isin(movieid_list)].loc[:, ["movieId", "original_title"]]

,movieId,original_title
4824,299,Ocean's Eleven
6918,3089,Red River
7488,31952,The Vampire Lovers
7938,2502,The Bourne Supremacy
11008,58,Pirates of the Caribbean: Dead Man's Chest


In [49]:
print(pred_rating)

[4.72816277 4.76283312 5.16273499 4.81649208 4.96605873]
